In [1]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.1 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import time
from collections import namedtuple

Firstly, we will download the famous dataset sst-2 for Stanford Sentiment Treebank. It is for sentimental analysis.

In [4]:
# Charger le dataset de sentimental analysis
dataset = load_dataset("sst2")

# Récupérer les features (X) et les étiquettes (y) du dataset
X = dataset["train"]["sentence"]
y = dataset["train"]["label"]

# Récupère seulement 5% des données pour le test
subset_size = len(X) // 20
X_test = X[:subset_size]
y_test = y[:subset_size]


# Afficher les informations sur l'ensemble de test
print("Test data:")
print("Number of samples:", len(X_test))
print("X : ", X_test[:5])
print("Y : ", y_test[:5])

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Test data:
Number of samples: 3367
X :  ['hide new secretions from the parental units ', 'contains no wit , only labored gags ', 'that loves its characters and communicates something rather beautiful about human nature ', 'remains utterly satisfied to remain the same throughout ', 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ']
Y :  [0, 0, 1, 0, 0]


Now, let download some models to test them

In [5]:
Metrics = namedtuple('Metrics', ('model_name', 'accuracy', 'precision', 'recall', 'f1', 'run_time', 'step_per_second', 'element_per_second'))

class AllMetrics:
  def __init__(self, batchSize, dataSize):
    self.batchSize = batchSize
    self.dataSize = dataSize
    self.step = dataSize / batchSize

    self.all_models_performances = list()

  def compute_new_metrics(self, name, y_test, predicted_labels, run_time):
    # Get the performances of the model
    accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels)
    recall = recall_score(y_test, predicted_labels)
    f1 = f1_score(y_test, predicted_labels)

    # Get the execution performances
    step_per_second = run_time / self.step
    element_per_second = run_time / self.dataSize

    # Add to tuple and append to list of models
    performance = Metrics(name, accuracy, precision, recall, f1, run_time, step_per_second, element_per_second)
    self.all_models_performances.append(performance)


  def display_performance_models(self):
    for model in self.all_models_performances:
      print("=========================")
      print("model : ", model.model_name)
      print("Accuracy:", model.accuracy)
      print("Precision:", model.precision)
      print("Recall:", model.recall)
      print("F1 Score:", model.f1)
      print("run_time:", model.run_time)
      print("step_per_second:", model.step_per_second)
      print("element_per_second:", model.element_per_second)


In [6]:
model_names = []
model_names.append("Kaludi/Reviews-Sentiment-Analysis")
model_names.append("distilbert-base-uncased-finetuned-sst-2-english")
model_names.append("bert-base-uncased")
model_names.append("Ibrahim-Alam/finetuning-roberta-base-on-sst2")
model_names.append("roberta-base")

In [8]:
batchSize = 32
allMetrics = AllMetrics(batchSize, len(X_test))

for name in model_names:
  # Display model name
  print(name)

  # Load the model and the tokenizer
  model = AutoModelForSequenceClassification.from_pretrained(name)
  tokenizer = AutoTokenizer.from_pretrained(name)
  if name == "Kaludi/Reviews-Sentiment-Analysis":
    max_length = 128
  else:
    max_length = tokenizer.model_max_length

  # Initialize predicted labels
  predicted_labels = torch.empty(0, dtype=torch.int64)

  # Measure the starting time
  start_time = time.process_time()

  # Run the model on the all data by group of 32
  for i in range(0, len(X_test), batchSize):
    if i%512 == 0:
      print("Step:", i)

    # Prétraiter les données
    batch = tokenizer(X_test[i:i+batchSize], padding=True, truncation=True, max_length=max_length, return_tensors="pt")

    # Faire les prédictions sur les données de test
    with torch.no_grad():
        outputs = model(**batch)
        predictions = F.softmax(outputs.logits, dim=1)
        predicted_labels = torch.cat((predicted_labels, torch.argmax(predictions, dim=1)))

  # Measure the ending time
  end_time = time.process_time()

  # Compute the delta time
  delta_time = end_time - start_time

  # Keep data in the class AllMetrics
  allMetrics.compute_new_metrics(name, y_test, predicted_labels, delta_time)

Kaludi/Reviews-Sentiment-Analysis


Step: 0
Step: 512
Step: 1024
Step: 1536
Step: 2048
Step: 2560
Step: 3072
distilbert-base-uncased-finetuned-sst-2-english


Step: 0
Step: 512
Step: 1024
Step: 1536
Step: 2048
Step: 2560
Step: 3072
bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Step: 0
Step: 512
Step: 1024
Step: 1536
Step: 2048
Step: 2560
Step: 3072
Ibrahim-Alam/finetuning-roberta-base-on-sst2


Step: 0
Step: 512
Step: 1024
Step: 1536
Step: 2048
Step: 2560
Step: 3072
roberta-base


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should pr

Step: 0
Step: 512
Step: 1024
Step: 1536
Step: 2048
Step: 2560
Step: 3072


In [9]:
allMetrics.display_performance_models()

model :  Kaludi/Reviews-Sentiment-Analysis
Accuracy: 0.8411048411048411
Precision: 0.7968468468468468
Recall: 0.9546681057744199
F1 Score: 0.8686471888043211
run_time: 626.335927477
step_per_second: 5.9527026074440155
element_per_second: 0.18602195648262548
model :  distilbert-base-uncased-finetuned-sst-2-english
Accuracy: 0.9878229878229878
Precision: 0.988673139158576
Recall: 0.9892066918510524
F1 Score: 0.98893984353925
run_time: 228.32516803400006
step_per_second: 2.1700045669997037
element_per_second: 0.06781264271874074
model :  bert-base-uncased
Accuracy: 0.5503415503415503
Precision: 0.5503415503415503
Recall: 1.0
F1 Score: 0.7099616858237547
run_time: 449.948192253
step_per_second: 4.276311895484407
element_per_second: 0.13363474673388773
model :  Ibrahim-Alam/finetuning-roberta-base-on-sst2
Accuracy: 0.9664389664389664
Precision: 0.964247598719317
Recall: 0.9751753912574204
F1 Score: 0.969680708344513
run_time: 462.0120550020001
step_per_second: 4.390966961705971
element_per_